In [23]:
import numpy as np
from bound_rewards_l import *
from lava_problem import main as lava_problem
import scipy.optimize as optimize
import matplotlib.pyplot as plt
import cvxpy as cvx

[-1.00269526 -0.34664925  0.74434933  1.56226311] 4.10229320763864


In [24]:
# Setups for lava problem
p_correct_vals = np.linspace(0.01, 0.99, 20) 
reward_xs = np.array([0.1]) 
reward_x = reward_xs[0]

# Function
def bounds(svec, p_correct = p_correct_vals[0]):
		nx, nu, ny, T, p0, px_x, py_x, R, R0_expected = lava_problem(['--p_correct', str(p_correct), '--reward_x', str(reward_x)])
		bound_f_inverse = compute_bound(l,n,svec, nx, nu, ny, T, p0, px_x, py_x, R, R0_expected) 
		return bound_f_inverse

# Constraint: from 0 to 2, each slope has to be larger
# So elements in the slope vector need to be in increasing order
def con(svec):
    out = []
    for i in range(len(svec)-1):
        out.append(svec[i+1]-svec[i])
    return out
def con1(svec):
    return l(0,n,svec)-1
cons = [{'type':'ineq','fun': con}]

# Print optimization progress
def callbackF(svec,status):
    print (svec,bounds(svec))

In [28]:
@ray.remote
def minimize(p_correct,svec):
	res = optimize.minimize(bounds,svec,method = 'trust-constr', constraints = cons, args = (p_correct), options={'disp':True})
	return[res.x, res.fun]

[-4.32718217 -2.24644184  2.2256411   5.30524678] 4.092235843810651


In [26]:
n = 2
s0 = [-1,-1,1,1]

In [ ]:
minimize(p_correct_vals[0],s0)

In [ ]:
import threading
# t0 = threading.Thread(target=minimize,args=(p_correct_vals[0],s0))
# t1 = threading.Thread(target=minimize,args=(p_correct_vals[1],s0))
# t0.start()
# t1.start()
# t0.join()
# t1.join()
stop_threads = True

In [29]:
import ray
ray.init()

[-4.21085945 -1.46885154  0.88852871  6.02997018] 4.123530671876245


2022-09-20 13:27:24,180	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.10
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


[-0.67116116 -0.5923559   0.74689066  1.33025612] 4.058070371932471


In [31]:
futures = [minimize.remote(p,s0) for p in p_correct_vals[0:3]]
print(ray.get(futures))

[-4.32718217 -2.24644184  2.2256411   5.30524678] 4.0922358436939765
[-4.22738194 -1.47522688  0.89046754  6.05210859] 4.123536307013382
[-1.3606551  -1.29531908  1.15825439  2.20053083] 4.055362383204825


(minimize pid=610550) /usr/lib/python3/dist-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
(minimize pid=610550)   warn('delta_grad == 0.0. Check if the approximated '
(minimize pid=610487) /usr/lib/python3/dist-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
(minimize pid=610487)   warn('delta_grad == 0.0. Check if the approximated '
(minimize pid=610467) /usr/lib/python3/dist-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better result

[-4.32718217 -2.24644184  2.2256411   5.30524678] 4.092235843635639
[-4.23562089 -1.47840595  0.89143434  6.06314793] 4.123539101816787
[-1.53604666 -1.51084628  1.27549493  2.43363442] 4.053884174865029
[-4.3262909  -2.24806414  2.22621492  5.30540399] 4.092169421639981
[-4.24384512 -1.48157933  0.89239942  6.07416755] 4.123541881634982
[-1.87720121 -1.8538728   1.47738867  2.86270665] 4.053712571848573
[-3.71880161 -3.35379199  2.61731239  5.41254618] 4.058262567912348
[-4.25205471 -1.48474707  0.89336277  6.08516755] 4.123544646602006
[-3.71880161 -3.35379199  2.61731239  5.41254618] 4.058262567912348
[-1.87720121 -1.8538728   1.47738867  2.86270665] 4.053712571848613
[-4.26024973 -1.48790918  0.89432442  6.09614802] 4.123547396844962
[-3.66095508 -3.57251727  2.70622636  5.48451108] 4.054307934627336
[-3.66095508 -3.57251727  2.70622636  5.48451108] 4.054307934627336
[-3.20681112 -3.16682767  2.25602579  4.5273152 ] 4.053743696895013
[-4.26843022 -1.49106569  0.89528436  6.10710902

In [ ]:
from dask.distributed import Client
client = Client(n_workers=2)
client

In [ ]:
tr = 20
opt_results = []
s0_guesses = [s0]
opt_results.append(minimize(p_correct_vals[0],s0)) # first iteration
s0_guesses.append(opt_results[0][0])

futures = []
for i in range(1,tr):
    future = client.submit(minimize,p_correct_vals[i],s0_guesses[-1])
    futures.append(future)



In [ ]:
tr = 3
opt_results = []
s0_guesses = [s0]
opt_results.append(minimize(p_correct_vals[0],s0)) # first iteration
s0_guesses.append(opt_results[0][0])

for i in range(1,tr):
    opt_result = minimize(p_correct_vals[i],s0_guesses[-1])
    opt_results.append(opt_result)

In [ ]:
# bounds([-1.0130594 , -1.01305833,  0.73804023,  1.29017034], 0.6289473684210526)
bounds([-1,-1,1,1], 0.6289473684210526)

In [ ]:
client.close()

In [ ]:
minimize(0.6289473684210526,[-1.18994735, -1.03399307,  0.7820952,   1.47360323])

In [ ]:

bounds([-3.53,1.18,1.18,1.18],0.6289473684210526)

In [ ]:
# Plot POMDP v picked tightest bounds
opt_data = np.load("results/lava_problem_optimal_results.npz")
p_correct_vals = opt_data['p_correct_vals']
opt_values = opt_data['opt_values']

# Plot
fig, ax = plt.subplots()
ax.plot(p_correct_vals, opt_values, '*--', label='POMDP', linewidth=0.5)

data = np.load("tightest_bounds.npz")
bounds = data['bounds']
ax.plot(p_correct_vals, bounds, 'o--', label='Tightest bounds', linewidth=1) 

ax.plot(p_correct_vals[0:5],[4.0531239449,3.9676793198,3.777793097,3.5682540424,3.4912805122],'+--',label='New results')

plt.xlabel('$p_{correct}$', fontsize=15)
plt.ylabel('Cumulative reward', fontsize=15)
plt.legend(fontsize=12, loc='lower right')
plt.ylim([0, 5.01])
plt.gcf().set_dpi(150)
plt.show() 

In [ ]:

y=np.array(opt_results)
np.savez('y.npz',y)

In [ ]:
data = np.load('y.npz')

In [ ]:
tight_data = np.load("tightest_bounds.npz")

In [ ]:
tight_data['bounds']

In [ ]:
[opt_results[j][0] for j in range(3)]